In [1]:
from process_framework.steps.sql import GetOrmQueryResult, GetTextQueryResult
from process_framework import Reference
from pandas import DataFrame, Series
from sqlalchemy import create_engine, URL, Engine, engine_from_config
import json
from pathlib import Path
from inflection import underscore
import os
assert (root := os.environ["WORKSPACE_ROOT"]), 'expected to find `WORKSPACE_ROOT` env var with path to workspace root'

In [2]:
url_args = json.loads(Path(root, 'secrets', 'research_reports_db.json').read_text())
url_args

{'drivername': 'mssql+pyodbc',
 'host': 'SVMSQL03',
 'database': 'Cfa_Local',
 'query': {'driver': 'ODBC Driver 17 for SQL Server'}}

In [3]:
url = URL.create(**url_args)
engine = create_engine(url)
engine

Engine(mssql+pyodbc://SVMSQL03/Cfa_Local?driver=ODBC+Driver+17+for+SQL+Server)

In [4]:
df_ref = Reference(DataFrame)

In [5]:
# try the different constructors
QUERY = 'SELECT TOP 10 * FROM tblReports'

for k, v in [('engine', dict(engine=engine)), ('engine, args',dict(engine=engine, url_create_kwargs=url_args)), ('args', dict(url_create_kwargs=url_args))]:
    print(k)
    try: get_result = GetTextQueryResult(df_ref, query=QUERY, index='reportUID', **v, column_mapper=underscore); print(get_result)
    except Exception as e: print(e)
    print('\n')

engine


engine, args
expected exactly one of `engine` and `url_create_kwargs`, got engine:True url_create_kwargs:True


args




In [6]:
# assign a DataFrame result to a Reference[DataFrame]
#   using a column_mapper, so updating `index` to match
ref = Reference(DataFrame)

QUERY = 'SELECT TOP 10 * FROM tblReports'
get_result = GetTextQueryResult(ref, query=QUERY, column_mapper=underscore, index='report_uid', engine=engine)
get_result.do()

ref.value

,series_uid,source_uid,title,date,embargoed,published,summary,pages,old_series_numbering,report_year,report_number,old_series_report_number,available_online
report_uid,,,,,,,,,,,,,
1,2,20,RUNNYMEDE BRIDGE. TREE-RING ANALYSIS,1986-04-01,False,True,,7,True,None,None,4847,True
2,2,20,"AMPHORAE FROM BANNAVENTA, NORTHAMPTONSHIRE",1986-04-01,False,True,,2,True,None,None,4844,True
3,2,20,STONE FROM THE MEDIEVAL LEVELS AT STAPLE GARDE...,1986-04-01,False,True,,1,True,None,None,4843,True
4,2,20,ANALYSIS OF NON-FERROUS METAL OBJECTS FROM MOR...,1985-12-01,False,True,,12,True,None,None,4724,True
5,2,0,A REPORT ON THE PALYNOLOGICAL DATING OF COAL S...,1972-07-01,False,True,A REPORT ON THE PALYNOLOGICAL DATING OF COAL S...,29,True,None,None,1497,True
6,2,20,"INTERIM SOIL REPORT ON WHERWELL, HANTS 1964",1970-04-10,False,True,,2,True,None,None,1498,True
7,2,11,"CHARCOAL FROM POXWELL, DORSET",1973-02-04,False,True,,1,True,None,None,1499,True
8,2,20,"BALDOCK, HERTS; ROMAN CEMETERY, TEXTILES",1973-01-01,False,True,,1,True,None,None,1500,True
9,2,20,"SOIL-FLOOR MATERIAL; HINTON HALL,HADDENHAM",1972-03-06,False,True,,1,True,None,None,1501,True


In [7]:
# assign one-column 'Series' result to a Reference[Series]
#   the `index='reportUID'` argument sets a column to the result's index, leaving one column
ref = Reference(Series)

QUERY = 'SELECT TOP 10 reportUID, title FROM tblReports'
get_result = GetTextQueryResult(ref, query=QUERY, index='reportUID', engine=engine)
get_result.do()

print(ref)
ref.value

Reference[Series]((10,), ['RUNNYMEDE BR...RING ANALYSIS', 'AMPHORAE FRO...THAMPTONSHIRE', 'STONE FROM T...NS,WINCHESTER', ...])


reportUID
1                  RUNNYMEDE BRIDGE. TREE-RING ANALYSIS
2            AMPHORAE FROM BANNAVENTA, NORTHAMPTONSHIRE
3     STONE FROM THE MEDIEVAL LEVELS AT STAPLE GARDE...
4     ANALYSIS OF NON-FERROUS METAL OBJECTS FROM MOR...
5     A REPORT ON THE PALYNOLOGICAL DATING OF COAL S...
6           INTERIM SOIL REPORT ON WHERWELL, HANTS 1964
7                         CHARCOAL FROM POXWELL, DORSET
8              BALDOCK, HERTS; ROMAN CEMETERY, TEXTILES
9            SOIL-FLOOR MATERIAL; HINTON HALL,HADDENHAM
10    A RECIRCULATING DEIONIZING/WASH SYSTEM FOR ARC...
Name: title, dtype: object

In [8]:

from sqlalchemy import ColumnElement, MetaData, Select, Table, Column, select, Integer


class AssignReportsByOrmQuery(GetOrmQueryResult):
    def get_query(self) -> Select:
        return select(
            self.reports.c.reportUID,
            self.reports.c.title
        )

    def get_in_column(self) -> ColumnElement:
        return self.reports.c.reportUID

    def populate_metadata(self, metadata: MetaData) -> None:
        self.reports = Table(
            'tblReports', metadata,
            Column('reportUID', Integer),
            Column('title')
        )


In [9]:
tbl = Reference(DataFrame)

getter = AssignReportsByOrmQuery(tbl, engine=engine, limit=7000)
getter.do()

tbl

Reference[DataFrame]((7000, 2), {0: {'reportUID': 1, 'title': 'RUNNYMEDE BR...RING ANALYSIS'}, 1: {'reportUID': 2, 'title': 'AMPHORAE FRO...THAMPTONSHIRE'}, 2: {'reportUID': 3, 'title': 'STONE FROM T...NS,WINCHESTER'}, ...})

In [10]:
_ids = tbl.value.reportUID.to_list()
len(_ids)

7000

In [11]:
tbl = Reference(DataFrame)

getter = AssignReportsByOrmQuery(tbl, engine=engine, _ids=Series(_ids).sample(n=5000).to_list())
print(getter.get_qualified_query())

SELECT "tblReports"."reportUID", "tblReports".title 
FROM "tblReports" JOIN "#TEMP_IDS" ON "tblReports"."reportUID" = "#TEMP_IDS"._id


In [12]:
getter.do()

tbl

Reference[DataFrame]((5000, 2), {0: {'reportUID': 1, 'title': 'RUNNYMEDE BR...RING ANALYSIS'}, 1: {'reportUID': 3, 'title': 'STONE FROM T...NS,WINCHESTER'}, 2: {'reportUID': 4, 'title': 'ANALYSIS OF ...ORPE, NORFOLK'}, ...})

In [13]:
# this falls over because our query has two columns (so, a DataFrame) to a Reference[Series]
ref = Reference(Series)

QUERY = 'SELECT TOP 10 title, date FROM tblReports'
get_result = GetTextQueryResult(ref, query=QUERY, engine=engine)
get_result.do()

ref

Exception: `assign_to` expects a `<class 'pandas.core.series.Series'>`, but `result` is <class 'pandas.core.frame.DataFrame'>